# 1. Overview

The notebook covers data preparation for ASML and its suppliers, focusing on combining weekly stock data and adding binary variables for exogenous events like COVID, geopolitical tensions, and tech regulations. It starts with data loading and merging steps, ensuring a consistent weekly frequency across datasets. After checking for missing values, the document introduces binary indicators to represent the occurrence of specific external events over time. Finally, the combined dataset is exported for further analysis, encapsulating all relevant market and geopolitical factors.

# 2. Data Understanding

## 2.1. Data Description

## 2.2. Import Necessary Libraries

In [1]:
import pandas as pd

## 2.3. Define global variables

In [2]:
input_data_asml = 'data/weekly_data_asml.csv'
input_data_suppliers = 'data/weekly_data_suppliers.csv'
output_data = 'data/weekly_data_combination.csv'

## 2.4. Functions

In [3]:
# Function to add exogenous variables to the future_dataframe
def add_exogenous_variables(weekly_data):
    """
    Input: future_df (DataFrame) - A DataFrame containing future dates in the 'ds' column.
    Output: future_df (DataFrame) - The same DataFrame with added binary columns for each exogenous event.
    Logic: Adds binary indicators for each specified geopolitical and economic event based on predefined date ranges.
    """
    
    # Define the date ranges for each exogenous event
    covid_start = '2020-01-01'
    covid_end = '2022-12-31'

    geopolitical_tension_start = '2022-02-01'
    geopolitical_tension_end = '2025-12-31'

    trade_sanctions_start = '2018-07-01'
    trade_sanctions_end = '2029-12-31'

    tech_regulation_start = '2020-06-01'
    tech_regulation_end = '2029-12-31'

    new_product_launch_start = '2023-12-01'
    new_product_launch_end = '2025-12-31'

    israel_gaza_conflict_start = '2023-10-07'
    israel_gaza_conflict_end = '2025-12-31'

    # COVID Period
    weekly_data['COVID_Period'] = ((weekly_data.index >= covid_start) & 
                                 (weekly_data.index <= covid_end)).astype(int)
    
    # Geopolitical Tensions (Ukraine Conflict)
    weekly_data['Geopolitical_Tension'] = ((weekly_data.index >= geopolitical_tension_start) & 
                                         (weekly_data.index <= geopolitical_tension_end)).astype(int)
    
    # US-China Trade War
    weekly_data['Trade_Sanctions'] = ((weekly_data.index >= trade_sanctions_start) & 
                                    (weekly_data.index <= trade_sanctions_end)).astype(int)
    
    # Tech Regulation (Export Restrictions)
    weekly_data['Tech_Regulation'] = ((weekly_data.index >= tech_regulation_start) & 
                                    (weekly_data.index <= tech_regulation_end)).astype(int)
    
    # New EUV Machine Launch (TWINSCAN EXE:5000)
    weekly_data['New_Product_Launch'] = ((weekly_data.index >= new_product_launch_start) & 
                                       (weekly_data.index <= new_product_launch_end)).astype(int)
    
    # Israel-Gaza Conflict
    weekly_data['Israel_Gaza_Conflict'] = ((weekly_data.index >= israel_gaza_conflict_start) & 
                                         (weekly_data.index <= israel_gaza_conflict_end)).astype(int)

## 2.5. Code

### 2.5.1. Looking at the datasets

**ASML**

In [4]:
weekly_data_asml = pd.read_csv(input_data_asml, sep=';')

# Ensure the 'Date' column is in datetime format
weekly_data_asml['Date'] = pd.to_datetime(weekly_data_asml['Date'])

# Set the 'Date' column as the index to make it a DatetimeIndex
weekly_data_asml.set_index('Date', inplace=True)

# Explicitly set the frequency of the Date index
weekly_data_asml = weekly_data_asml.asfreq('W')

# Rename the 'Close' column to 'Close_ASML'
weekly_data_asml.rename(columns={'Close': 'Close_ASML'}, inplace=True)

weekly_data_asml

,Close_ASML
Date,
2013-01-13,56.544903
2013-01-20,57.202209
2013-01-27,62.105244
2013-02-03,66.857285
2013-02-10,67.496826
...,...
2024-10-13,832.022278
2024-10-20,721.570862
2024-10-27,710.037903


**Suppliers**

In [5]:
weekly_data_suppliers = pd.read_csv(input_data_suppliers, sep=';')

# Ensure the 'Date' column is in datetime format
weekly_data_suppliers['Date'] = pd.to_datetime(weekly_data_suppliers['Date'])

# Set the 'Date' column as the index to make it a DatetimeIndex
weekly_data_suppliers.set_index('Date', inplace=True)

# Explicitly set the frequency of the Date index
weekly_data_suppliers = weekly_data_suppliers.asfreq('W')

# Drop certain columns
weekly_data_suppliers = weekly_data_suppliers.drop(['Date.1'], axis=1)

weekly_data_suppliers

,Close_Carl_Zeiss_Meditec,Close_Lam_Research_Corporation,Close_Tokyo_Electron_Limited,Close_Taiwan_Semiconductor_Manufacturing_Company,Close_Samsung_Electronics,Close_Intel_Corporation,Weekday,Month,Quarter
Date,,,,,,,,,
2019-01-06,75.651291,8.299143,18.953333,30.410763,32299.406250,40.518044,3.0,1.0,1.0
2019-01-13,76.849060,8.656210,20.006666,30.967327,34017.457031,41.318424,2.0,1.0,1.0
2019-01-20,82.406708,8.441608,21.273333,31.341261,35606.660156,41.714314,2.0,1.0,1.0
2019-01-27,82.406708,9.049646,22.076667,32.232623,36723.394531,41.400187,2.0,1.0,1.0
2019-02-03,86.019196,10.193889,23.473333,32.558723,39644.078125,40.552475,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
2024-10-20,67.094002,74.139999,81.540001,192.210007,59500.000000,22.660000,2.0,10.0,4.0
2024-10-27,67.529999,73.000000,76.709999,200.860001,57700.000000,22.400000,2.0,10.0,4.0
2024-11-03,65.550003,75.449997,77.500000,194.479996,59100.000000,22.900000,2.0,10.0,4.0


Let's create a weekly_data dataframe from both weekly_data_asml and weekly_data_suppliers

In [6]:
weekly_data = weekly_data_asml.join(weekly_data_suppliers, how='inner')
weekly_data

,Close_ASML,Close_Carl_Zeiss_Meditec,Close_Lam_Research_Corporation,Close_Tokyo_Electron_Limited,Close_Taiwan_Semiconductor_Manufacturing_Company,Close_Samsung_Electronics,Close_Intel_Corporation,Weekday,Month,Quarter
Date,,,,,,,,,,
2019-01-06,146.401947,75.651291,8.299143,18.953333,30.410763,32299.406250,40.518044,3.0,1.0,1.0
2019-01-13,152.422821,76.849060,8.656210,20.006666,30.967327,34017.457031,41.318424,2.0,1.0,1.0
2019-01-20,149.290100,82.406708,8.441608,21.273333,31.341261,35606.660156,41.714314,2.0,1.0,1.0
2019-01-27,153.062531,82.406708,9.049646,22.076667,32.232623,36723.394531,41.400187,2.0,1.0,1.0
2019-02-03,164.662170,86.019196,10.193889,23.473333,32.558723,39644.078125,40.552475,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
2024-10-13,832.022278,72.779999,82.059998,86.790001,186.050003,59800.000000,23.320000,2.0,10.0,4.0
2024-10-20,721.570862,67.094002,74.139999,81.540001,192.210007,59500.000000,22.660000,2.0,10.0,4.0
2024-10-27,710.037903,67.529999,73.000000,76.709999,200.860001,57700.000000,22.400000,2.0,10.0,4.0


Display the total number of nulls per column


In [7]:
nulls_per_column = weekly_data.isnull().sum()

nulls_per_column

Close_ASML                                          0
Close_Carl_Zeiss_Meditec                            0
Close_Lam_Research_Corporation                      0
Close_Tokyo_Electron_Limited                        0
Close_Taiwan_Semiconductor_Manufacturing_Company    0
Close_Samsung_Electronics                           0
Close_Intel_Corporation                             0
Weekday                                             0
Month                                               0
Quarter                                             0
dtype: int64

### 2.5.2 Add binary variables

In [8]:
# Apply the function to weekly_data
add_exogenous_variables(weekly_data)

In [9]:
weekly_data

,Close_ASML,Close_Carl_Zeiss_Meditec,Close_Lam_Research_Corporation,Close_Tokyo_Electron_Limited,Close_Taiwan_Semiconductor_Manufacturing_Company,Close_Samsung_Electronics,Close_Intel_Corporation,Weekday,Month,Quarter,COVID_Period,Geopolitical_Tension,Trade_Sanctions,Tech_Regulation,New_Product_Launch,Israel_Gaza_Conflict
Date,,,,,,,,,,,,,,,,
2019-01-06,146.401947,75.651291,8.299143,18.953333,30.410763,32299.406250,40.518044,3.0,1.0,1.0,0,0,1,0,0,0
2019-01-13,152.422821,76.849060,8.656210,20.006666,30.967327,34017.457031,41.318424,2.0,1.0,1.0,0,0,1,0,0,0
2019-01-20,149.290100,82.406708,8.441608,21.273333,31.341261,35606.660156,41.714314,2.0,1.0,1.0,0,0,1,0,0,0
2019-01-27,153.062531,82.406708,9.049646,22.076667,32.232623,36723.394531,41.400187,2.0,1.0,1.0,0,0,1,0,0,0
2019-02-03,164.662170,86.019196,10.193889,23.473333,32.558723,39644.078125,40.552475,2.0,1.0,1.0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-13,832.022278,72.779999,82.059998,86.790001,186.050003,59800.000000,23.320000,2.0,10.0,4.0,0,1,1,1,1,1
2024-10-20,721.570862,67.094002,74.139999,81.540001,192.210007,59500.000000,22.660000,2.0,10.0,4.0,0,1,1,1,1,1
2024-10-27,710.037903,67.529999,73.000000,76.709999,200.860001,57700.000000,22.400000,2.0,10.0,4.0,0,1,1,1,1,1


# 3. Export

In [10]:
# Export the weekly_data_suppliers to a CSV file
weekly_data.to_csv(output_data, index=True, sep=';')